## This notebook uses Daily Classifier

# Prep

In [ ]:
import pandas as pd
from tqdm import tqdm
from imblearn.over_sampling import SMOTE
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import f1_score, precision_score
import numpy as np
from xgboost import XGBClassifier
import re

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def eval_predictions(true, predicted):
    geometric_score = geometric_mean_score(
        true,
        predicted,
        average="macro",
    )

    # Calculate the f1 score
    f1 = f1_score(
        true,
        predicted,
        average="macro",
    )

    precision = precision_score(
        true, 
        predicted,
        average='macro'
    )

    # print(f"Geometric Mean Score: {geometric_score:.4f}")
    # print(f"F1 Score: {f1:.4f}")
    # print(f"Precision Score: {precision:.4f}")

    return geometric_score, f1, precision

In [ ]:
def get_np_array(df, columns):
    arr = np.stack(df[columns[0]].to_numpy())
    for c in range(1,len(columns)):
        arr = np.concatenate([arr,np.stack(df[columns[c]].to_numpy())],1)
    return arr

def parse_timestamp_list(s):
    # find all datetime strings inside Timestamp('...')
    matches = re.findall(r"Timestamp\('([^']+)'\)", s)
    # convert each to pd.Timestamp
    return [pd.to_datetime(m) for m in matches]

# Load

In [ ]:
df = pd.read_csv("../data/sites_ABCD_NewFeatures.csv", index_col=0)
df['date'] = pd.to_datetime(df['date'])
df['timestamp'] = pd.to_datetime(df['timestamp'])

working_hours = [10,11,12,13,14,15,16,17]

df = df[(df["hour"].isin(working_hours))].groupby(["site", "date"]).aggregate(list).sort_index().reset_index()

minf = df['demand_response'].apply(min)
maxf = df['demand_response'].apply(max)
df["DayResponse"] = np.where(
    (minf==0) & (maxf==1),
    1,
    np.where(
        (minf==-1) & (maxf==0),
        -1,
        np.where(
            (minf==-1) & (maxf==1),
            2,
            0
        )
        )
    )


df_ABCD = df.copy()
del df

In [ ]:
df_ABC_interval = pd.read_csv("../data/sites_ABC.csv")[['Site', 'Timestamp_Local', 'Demand_Response_Flag']]
df_DEF_interval = pd.read_csv("../data/sites_DEF.csv")[['Site', 'Timestamp_Local', 'Demand_Response_Flag']]
df_ABCD_interval = pd.concat([df_ABC_interval, df_DEF_interval[df_DEF_interval['Site']=='siteD']]).reset_index(drop=True)
df_ABCD_interval['Timestamp_Local'] = pd.to_datetime(df_ABCD_interval['Timestamp_Local'])

### Try One Feature Set

In [ ]:
base_features = [ # - F1 | A: 0.6681 | B: 0.7033| C: 0.5943| D: 0.7796 | Mean: 0.6863
  'temp_corr_dev', 
  'power_zscore_sh',
  'power_zscore_sh_diff_t',
  'power_zscore_sh_diff_wdt',
  'power_zscore_sh_peek_diff',
  'power_zscore_sh_diff',
  'power_zscore_sh_peek_diff_t',
  'power_zscore_sh_hourly_std',
  'power_share_zscore_sh',
  'power_share_zscore_sh_diff',
  'power_share_zscore_sh_diff_t',
  'power_share_zscore_sh_diff_wdt',
  'power_share_zscore_sh_peek_diff',
  'power_share_zscore_st_hourly_std'
  ]

site_col, all_feat_col, new_feat_col, val_cnts_col, geom_col, f1_col, pr_col = [], [], [], [], [], [], []

combined_feats = base_features

for site in ['A', 'B', 'C', 'D']:
    
    tr_ixs = df_ABCD['site']!=f"site{site}"
    x_tr = get_np_array(df_ABCD.loc[tr_ixs], combined_feats)
    y_tr = df_ABCD.loc[tr_ixs, 'DayResponse'].to_numpy() +1

    x_te = get_np_array(df_ABCD.loc[~tr_ixs], combined_feats)

    # Train & Predict
    smote = SMOTE(random_state=94)
    x_tr_bal, y_tr_bal = smote.fit_resample(x_tr, y_tr)
    xgb = XGBClassifier()
    xgb.fit(x_tr_bal,y_tr_bal)
    preds = xgb.predict(x_te)

    # Post Process
    df_ABCD.loc[~tr_ixs, 'Pred'] = preds-1
    df_ABCD.loc[(~tr_ixs) & df_ABCD["month"].apply(lambda x: x[0]).isin([3,4,5,9,10,11]), 'Pred'] = 0 # No preds in shoulder seasons

    # Map to Interval
    neg1_start_ts = 9
    pos1_start_ts = 9
    pos2_start_ts = 2

    df_ABCD['Pred_Interval'] = df_ABCD['Pred'].apply(
        lambda x:
        [0]*(neg1_start_ts-1) + [-1]*(32-neg1_start_ts+1) if x==-1 else
        [0]*32 if x==0  else
        [0]*(pos1_start_ts-1) + [1]*(32-pos1_start_ts+1) if x==1 else
        [0]*(pos2_start_ts-1) + [1]*(8-pos2_start_ts+1) + [-1]*(24)
    )

    expanded_rows = []
    for _, row in df_ABCD.loc[~tr_ixs].iterrows():
        row_site = row['site']
        row_ts = row['timestamp']
        row_preds = row['Pred_Interval']
        for ts, pred in zip(row_ts, row_preds):
            expanded_rows.append({'Site':row_site, 'Timestamp_Local': ts, 'Pred_Interval': pred})

    expanded_df = pd.DataFrame(expanded_rows)
    expanded_df["Timestamp_Local"] = pd.to_datetime(expanded_df["Timestamp_Local"])
    expanded_df['Site'] = f"site{site}"

    df_ABCD_interval['Pred_Interval'] = 0
    df_ABCD_interval = pd.merge(df_ABCD_interval[['Site', 'Timestamp_Local', 'Demand_Response_Flag']], expanded_df, how='outer', left_on=["Site", "Timestamp_Local"], right_on=["Site", "Timestamp_Local"]).fillna(0)

    ixs_intvl = df_ABCD_interval['Site']==f"site{site}"

    gm, f1, pr = eval_predictions(
        true=df_ABCD_interval.loc[ixs_intvl,'Demand_Response_Flag'],
        predicted=df_ABCD_interval.loc[ixs_intvl,'Pred_Interval']
    )

    site_col.append(f"site{site}")
    all_feat_col.append(combined_feats)
    # new_feat_col.append(new_feats)
    val_cnts_col.append(df_ABCD.loc[~tr_ixs, 'Pred'].value_counts())
    geom_col.append(gm)
    f1_col.append(f1)
    pr_col.append(pr)

    if site=='D': print(f"- F1 | A: {f1_col[-4]:.4f} | B: {f1_col[-3]:.4f}| C: {f1_col[-2]:.4f}| D: {f1_col[-1]:.4f} | Mean: {np.mean(f1_col[-4:]):.4f}")

### Iterate Through Feature Sets

In [ ]:
non_feature_cols = ['site','timestamp','demand_response','demand_response_capacity','date','busday','time','minute','hour','quarter_hour','week','working_hours']
all_features = [col for col in df_ABCD.columns if col not in non_feature_cols + ['demand_response','demand_response_capacity','day_response', 'DayResponse','Pred','Pred_Interval']]

In [ ]:
base_features = ['temp_corr_dev', 'power_zscore_sh', 'power_zscore_sh_diff_t', 'power_zscore_sh_diff_wdt', 'power_zscore_sh_peek_diff', 'power_zscore_sh_diff', 'power_zscore_sh_peek_diff_t', 'power_zscore_sh_hourly_std', 'power_share_zscore_sh', 'power_share_zscore_sh_diff', 'power_share_zscore_sh_diff_t', 'power_share_zscore_sh_diff_wdt', 'power_share_zscore_sh_peek_diff', 'power_share_zscore_st_hourly_std', 'power_zscore_sh_peek4_diff', 'power_zscore_sh_lag4_diff', 'power_share_zscore_sh_peek4_diff', 'power_share_zscore_sh_lag4_diff']

In [ ]:
def evaluate_features(df, df_interval, feature_list, target="DayResponse", model=None):
    df_ABCD = df.copy()
    df_ABCD_interval = df_interval.copy()
    geom_scores, f1_scores, pr_scores = [], [], []
    for site in ['A', 'B', 'C', 'D']:
        
        tr_ixs = df_ABCD['site']!=f"site{site}"
        x_tr = get_np_array(df_ABCD.loc[tr_ixs], feature_list)
        y_tr = df_ABCD.loc[tr_ixs, 'DayResponse'].to_numpy() +1

        x_te = get_np_array(df_ABCD.loc[~tr_ixs], feature_list)

        # Train & Predict
        smote = SMOTE(random_state=94)
        x_tr_bal, y_tr_bal = smote.fit_resample(x_tr, y_tr)
        model.fit(x_tr_bal,y_tr_bal)
        preds = model.predict(x_te)

        # Post Process
        df_ABCD.loc[~tr_ixs, 'Pred'] = preds-1
        df_ABCD.loc[(~tr_ixs) & df_ABCD["month"].apply(lambda x: x[0]).isin([3,4,5,9,10,11]), 'Pred'] = 0 # No preds in shoulder seasons

        # Map to Interval
        neg1_start_ts = 9
        pos1_start_ts = 9
        pos2_start_ts = 2

        df_ABCD['Pred_Interval'] = df_ABCD['Pred'].apply(
            lambda x:
            [0]*(neg1_start_ts-1) + [-1]*(32-neg1_start_ts+1) if x==-1 else
            [0]*32 if x==0  else
            [0]*(pos1_start_ts-1) + [1]*(32-pos1_start_ts+1) if x==1 else
            [0]*(pos2_start_ts-1) + [1]*(8-pos2_start_ts+1) + [-1]*(24)
        )

        expanded_rows = []
        for _, row in df_ABCD.loc[~tr_ixs].iterrows():
            row_site = row['site']
            row_ts = row['timestamp']
            row_preds = row['Pred_Interval']
            for ts, pred in zip(row_ts, row_preds):
                expanded_rows.append({'Site':row_site, 'Timestamp_Local': ts, 'Pred_Interval': pred})

        expanded_df = pd.DataFrame(expanded_rows)
        expanded_df["Timestamp_Local"] = pd.to_datetime(expanded_df["Timestamp_Local"])
        expanded_df['Site'] = f"site{site}"

        df_ABCD_interval['Pred_Interval'] = 0
        df_ABCD_interval = pd.merge(df_ABCD_interval[['Site', 'Timestamp_Local', 'Demand_Response_Flag']], expanded_df, how='outer', left_on=["Site", "Timestamp_Local"], right_on=["Site", "Timestamp_Local"]).fillna(0)

        ixs_intvl = df_ABCD_interval['Site']==f"site{site}"

        gm, f1, pr = eval_predictions(
            true=df_ABCD_interval.loc[ixs_intvl,'Demand_Response_Flag'],
            predicted=df_ABCD_interval.loc[ixs_intvl,'Pred_Interval']
        )
        geom_scores.append(gm)
        f1_scores.append(f1)
        pr_scores.append(pr)

    return round(np.mean(geom_scores), 4), round(np.mean(f1_scores), 4), round(np.mean(pr_scores), 4)

In [ ]:
def feature_selection(df, df_interval, model, starting_features, features, target="DayResponse"):
    results = []

    # Baseline with starting features
    baseline_score = evaluate_features(df, df_interval, starting_features, target=target, model=model)[1]
    print(f"Baseline score: {baseline_score:.4f}")

    for feat in tqdm(features, desc="Testing features", unit="feat"):
        if feat not in starting_features:
            test_feats = starting_features + [feat]
            # print(f"Testing feature: {feat}")

            score = evaluate_features(df, df_interval, test_feats, target=target, model=model)[1]
            improvement = score - baseline_score

            results.append({
                "feature": feat,
                "score": score,
                "improvement": improvement
            })

    results_df = pd.DataFrame(results).sort_values("improvement",ascending=False)
    return results_df

def feature_ablation(df, df_interval, model, feature_list, dnt_list, target='DayResponse'):
    results = []

    # Baseline with all features
    baseline_score = evaluate_features(df, df_interval, feature_list, target=target, model=model)[1]
    print(f"Baseline score: {baseline_score:.4f}")

    features_to_test = [f for f in feature_list if f not in dnt_list]
    # Iterate by removing each feature once
    for feat in tqdm(features_to_test, desc="Ablating features", unit="feat"):
        reduced_feats = [f for f in feature_list if f != feat]
        score = evaluate_features(df, df_interval, reduced_feats, target=target, model=model)[1]
        change = score - baseline_score

        results.append({
            "feature": feat,
            "score": score,
            "change_vs_full": change
        })

    results_df = pd.DataFrame(results).sort_values("change_vs_full",ascending=False)
    return results_df

In [ ]:
test_model = XGBClassifier(
    random_state=42, 
    # n_estimators=100, 
    # learning_rate=0.05,
    # subsample=0.8, 
    # colsample_bytree=0.8
)

In [ ]:
base_features = ['temp_corr_dev', 'power_zscore_sh', 'power_zscore_sh_diff_t', 'power_zscore_sh_diff_wdt', 'power_zscore_sh_peek_diff', 'power_zscore_sh_diff', 'power_zscore_sh_peek_diff_t', 'power_zscore_sh_hourly_std', 'power_share_zscore_sh', 'power_share_zscore_sh_diff', 'power_share_zscore_sh_diff_t', 'power_share_zscore_sh_diff_wdt', 'power_share_zscore_sh_peek_diff', 'power_share_zscore_st_hourly_std', 'power_zscore_sh_peek4_diff', 'power_zscore_sh_lag4_diff', 'power_share_zscore_sh_peek4_diff', 'power_share_zscore_sh_lag4_diff', 'power_share_zscore_st_peek4_diff_t', 'season', 'month'] 

In [ ]:
stats = evaluate_features(df_ABCD, df_ABCD_interval, base_features, target="DayResponse", model=test_model)
print(stats)

In [ ]:
bad_features = ['power_zscore_st_lag4_diff', 'power_share_zscore_mt_peek_diff_t', 'power_share_std_st', 'power_zscore_st_lag4_diff_t', 'mean_usg_residual_zscore_mt_peek4_diff', 'mean_usg_residual_zscore_st_lag4_diff_t', 'irr_corr_dev', 'power_share_zscore_st_peek_diff', 'power_share_zscore_st_peek4_diff', 'mean_usg_residual', 'power_share_std_sh', 'mean_usg_residual_zscore_sh_lag_diff', 'power_share_mean_mt', 'power_zscore_mh_diff_t', 'mean_usg_residual_zscore_mh_peek4_diff_t', 'power_share_zscore_mh_hourly_std', 'mean_usg_residual_zscore_mt_lag4_diff', 'mean_usg_residual_mean_st', 'power_share_zscore_mh_peek_diff_t', 'power_zscore_sh_lag_diff', 'mean_usg_residual_zscore_sh_diff_t', 'mean_usg_residual_zscore_mt_peek4_diff_t', 'mean_usg_residual_mean_sh', 'power_share_zscore_mt_peek_diff', 'power_zscore_mt_lag4_diff', 'power_share_zscore_mh_lag4_diff', 'power_zscore_sh_peek4_diff_t', 'power_share_zscore_st_lag4_diff_t', 'power_zscore_mt_peek_diff_t', 'power_mean_mt', 'power_zscore_mt', 'mean_usg_residual_zscore_mh_hourly_std', 'power_zscore_mh', 'mean_usg_residual_std_sh', 'power_zscore_mt_hourly_std', 'power_zscore_mt_peek_diff', 'mean_usg_residual_zscore_mh_peek_diff_t', 'power_share_zscore_mh_lag4_diff_t', 'mean_usg_residual_zscore_mt', 'power_share_zscore_mh_peek4_diff_t', 'power_share_zscore_mh_peek_diff', 'power_share_zscore_mt_diff_t', 'mean_usg_residual_zscore_mh_lag4_diff_t', 'power_share_std_mh', 'mean_usg_residual_zscore_mt_diff_t', 'mean_usg_residual_zscore_sh_peek_diff_t', 'mean_usg_residual_zscore_mh_diff_wdt', 'power_share_zscore_mt_lag4_diff_t', 'power_share_std_mt', 'mean_usg_residual_mean_mt', 'day_of_week', 'power_zscore_mt_diff_t', 'mean_usg_residual_zscore_sh_peek4_diff', 'mean_usg_residual_zscore_st_diff_wdt', 'power_zscore_st_diff_wdt', 'power_share_zscore_sh_lag_diff', 'power_zscore_mh_peek_diff', 'power_share_zscore_mh_peek4_diff', 'power_share', 'mean_usg_residual_zscore_mh_diff', 'power_zscore_st_diff', 'mean_usg_residual_zscore_st_diff', 'mean_usg_residual_zscore_mh_lag_diff', 'mean_usg_residual_zscore_mh', 'power_share_mean_st', 'mean_usg_residual_zscore_st_lag4_diff', 'power_share_zscore_mt_diff', 'mean_usg_residual_mean_mh', 'power_share_zscore_mt_lag_diff_t', 'temp', 'power_zscore_st_peek_diff_t', 'mean_usg_residual_zscore_st_peek_diff_t', 'mean_usg_residual_zscore_mt_peek_diff', 'mean_usg_residual_zscore_sh_diff', 'power_share_zscore_st', 'mean_usg_residual_zscore_sh_peek4_diff_t', 'power_share_zscore_sh_hourly_std', 'power_zscore_mt_peek4_diff_t', 'power_share_zscore_sh_lag_diff_t', 'mean_usg_residual_std_mt', 'power_std_mt', 'power_share_zscore_mh_lag_diff', 'power_zscore_sh_lag4_diff_t', 'mean_usg_residual_zscore_sh_lag_diff_t', 'power_zscore_sh_lag_diff_t', 'power_std_st', 'mean_usg_residual_std_st', 'power_share_zscore_st_lag_diff_t', 'power_zscore_mh_lag4_diff_t', 'power_std_mh', 'power_share_zscore_mt_lag_diff', 'power_share_zscore_mh_diff_t', 'power_zscore_mh_peek4_diff_t', 'power_share_zscore_mh_lag_diff_t', 'week', 'mean_usg_residual_zscore_sh', 'power_zscore_mh_diff', 'mean_usg_residual_zscore_mt_lag_diff_t', 'power_zscore_mt_lag_diff_t', 'power_zscore_st', 'mean_usg_residual_zscore_st', 'power_mean_mh', 'power_mean_st', 'power_share_zscore_st_peek_diff_t', 'irr', 'irr_power_corr', 'power_std_sh', 'mean_usg_residual_std_mh', 'power_mean_sh', 'power']

In [ ]:
feature_results = feature_selection(df_ABCD, df_ABCD_interval, test_model, base_features, [f for f in all_features if f not in base_features + bad_features])

In [ ]:
selected_features = base_features.copy()
print(len(selected_features))

In [ ]:
# 0.7641
selected_features = ['temp_corr_dev', 'power_zscore_sh', 'power_zscore_sh_diff_t', 'power_zscore_sh_diff_wdt', 'power_zscore_sh_peek_diff', 'power_zscore_sh_diff', 'power_zscore_sh_peek_diff_t', 'power_zscore_sh_hourly_std', 'power_share_zscore_sh', 'power_share_zscore_sh_diff', 'power_share_zscore_sh_diff_t', 'power_share_zscore_sh_diff_wdt', 'power_share_zscore_sh_peek_diff', 'power_share_zscore_st_hourly_std', 'power_zscore_sh_peek4_diff', 'power_zscore_sh_lag4_diff', 'power_share_zscore_sh_peek4_diff', 'power_share_zscore_sh_lag4_diff', 'power_share_zscore_st_peek4_diff_t', 'season', 'month', 'power_share_zscore_sh_lag4_diff_t', 'power_share_zscore_mt_hourly_std', 'power_share_zscore_sh_peek4_diff_t', 'power_zscore_st_lag_diff_t', 'power_zscore_st_peek4_diff', 'mean_usg_residual_zscore_sh_diff_wdt', 'power_share_mean_sh', 'power_zscore_mh_peek_diff_t', 'mean_usg_residual_zscore_sh_lag4_diff', 'power_share_zscore_mh_diff', 'mean_usg_residual_zscore_mt_peek_diff_t', 'power_share_zscore_st_lag4_diff', 'mean_usg_residual_zscore_mt_diff', 'power_zscore_mh_lag_diff_t', 'power_share_mean_mh', 'mean_usg_residual_zscore_mh_peek_diff', 'power_zscore_st_peek_diff', 'power_zscore_mt_peek4_diff', 'mean_usg_residual_zscore_st_peek_diff'] 
print(len(selected_features))

In [ ]:
# bad_features = feature_results[feature_results['improvement'] < -0.02]['feature'].to_list()
# print(bad_features)

In [ ]:
score = 0.7518
while True:
    feature_results = feature_selection(df_ABCD, df_ABCD_interval, test_model, selected_features, [f for f in all_features if f not in selected_features + bad_features])
    new_ft = feature_results['feature'].iloc[0]
    new_score = feature_results['score'].iloc[0]
    score = new_score
    selected_features = selected_features + [new_ft]
    print(new_score, new_ft)
    print(selected_features, '\n')

In [ ]:
score = 0.7641
while True:
    feature_results = feature_ablation(df_ABCD, df_ABCD_interval, test_model, selected_features, dnt_list=[])
    new_ft = feature_results['feature'].iloc[0]
    new_score = feature_results['score'].iloc[0]
    score = new_score
    selected_features = [f for f in selected_features if f != new_ft]
    print(new_score, new_ft)
    print(selected_features, '\n')